In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, to_date, regexp_replace
from pyspark.sql.types import IntegerType, DecimalType, DateType

# Inicializar a sessão do Spark
spark = SparkSession.builder.appName("DataProcessing").getOrCreate()

def clean_and_transform_employees(file_path):
    """
    Limpa e transforma os dados dos funcionários, removendo duplicatas, espaços em branco, e garantindo tipos de dados corretos.
    
    Args:
        file_path (str): Caminho para o arquivo JSON contendo os dados dos funcionários.
    
    Returns:
        None
    """
    # Lê o arquivo JSON
    df = spark.read.option("multiline", "true").json(file_path)
    
    if "id" not in df.columns:
        print("A coluna 'id' não foi encontrada no DataFrame.")
        return
    
    # Remover espaços em branco
    df = df.withColumn("name", trim(col("name")))
    
    # Remover duplicatas
    df = df.dropDuplicates(['id'])
    
    # Converter 'id' para Integer
    df = df.withColumn("id", col("id").cast(IntegerType()))
    
    # Escrever os dados transformados no formato Parquet
    df.write.mode('overwrite').parquet("/mnt/processing/employees_data.parquet")
    print("Dados de funcionários processados e salvos na camada processing.")

def clean_and_transform_sales(file_path):
    """
    Limpa e transforma os dados de vendas, removendo duplicatas, garantindo tipos de dados corretos e formatando datas.
    
    Args:
        file_path (str): Caminho para o arquivo CSV contendo os dados de vendas.
    
    Returns:
        None
    """
    df = spark.read.option("header", "true").csv(file_path)
    
    # Remover duplicatas
    df = df.dropDuplicates(['id_venda'])
    
    # Remover espaços em branco
    df = df.withColumn("id_funcionario", trim(col("id_funcionario")))
    df = df.withColumn("id_categoria", trim(col("id_categoria")))
    
    # Converter 'id_funcionario' e 'id_categoria' para Integer
    df = df.withColumn("id_venda", col("id_venda").cast(IntegerType()))
    df = df.withColumn("id_funcionario", col("id_funcionario").cast(IntegerType()))
    df = df.withColumn("id_categoria", col("id_categoria").cast(IntegerType()))
    
    # Garantir que a coluna 'data_venda' está no formato de data
    df = df.withColumn("data_venda", to_date(col("data_venda"), "yyyy-MM-dd"))
    
    # Converter 'venda' para Integer (ou Decimal, se necessário)
    df = df.withColumn("venda", regexp_replace(col("venda"), ",", "").cast(DecimalType(10, 2)))
    
    # Escrever os dados transformados no formato Parquet
    df.write.mode('overwrite').parquet("/mnt/processing/sales_data.parquet")
    print("Dados de vendas processados e salvos na camada processing.")

def clean_and_transform_categories(file_path):
    """
    Limpa e transforma os dados de categorias, removendo duplicatas e espaços em branco.
    
    Args:
        file_path (str): Caminho para o arquivo Parquet contendo os dados de categorias.
    
    Returns:
        None
    """
    df = spark.read.parquet(file_path)
    
    # Remover duplicatas
    df = df.dropDuplicates(['id'])
    
    # Remover espaços em branco
    df = df.withColumn("nome_categoria", trim(col("nome_categoria")))
    
    # Converter 'id' para Integer
    df = df.withColumn("id", col("id").cast(IntegerType()))
    
    # Escrever os dados transformados no formato Parquet
    df.write.mode('overwrite').parquet("/mnt/processing/categories_data.parquet")
    print("Dados de categorias processados e salvos na camada processing.")

# Caminhos para os arquivos na landing
employees_json_path = "/mnt/landing/employees_data.json"
sales_csv_path = "/mnt/landing/sales_data.csv"
categories_parquet_path = "/mnt/landing/categoria.parquet"

# Chamar as funções de processamento
clean_and_transform_employees(employees_json_path)
clean_and_transform_sales(sales_csv_path)
clean_and_transform_categories(categories_parquet_path)


In [0]:
# Listar arquivos no container 'processing'
files = dbutils.fs.ls("/mnt/processing/")
for file in files:
    print(file.path)